In [ ]:
 !pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.3 MB/s 


In [ ]:
!pip install transformers -q

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.7 MB/s 
     |████████████████████████████████| 895 kB 39.1 MB/s 
     |████████████████████████████████| 636 kB 46.4 MB/s 


In [ ]:
!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  12854      0 --:--:-- --:--:-- --:--:-- 12854


In [ ]:
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

Updating... This may take around 2 minutes.
Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
\ [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
| [1 files][119.5 MiB/119.5 MiB]                                                
Operation completed over 1 objects/119.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
- [1 files][  2.3 MiB/  2.3 MiB]                                                
Operation co

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.6.0


BaseException: ignored

In [ ]:
!pip install pytorch_lightning==0.7.5

     |████████████████████████████████| 233 kB 4.0 MB/s 
     |████████████████████████████████| 829 kB 45.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ee9af18a51e92fee90267141b6ec2fe29d55401a14919f15b0a9cc909ecdb87e
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [ ]:
import transformers 
print(transformers.__version__)

4.9.2


In [ ]:
import torch 
print(torch.__version__)

1.6.0a0+bf2bbd9


In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Mon Aug 23 16:42:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(str(device))

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

cpu


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt',truncation=True)
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt',truncation=True)

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            #print(str(outputs))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=200, 
                num_beams=2,
                repetition_penalty=3.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
!pip install rouge-score

In [ ]:
import pandas as pd
tokenizer = T5Tokenizer.from_pretrained("t5-small")
  
  
df = pd.read_csv('/content/SummaryDataSetCustom.tsv',encoding='latin-1',delimiter="\t")
df = df[['STORY','SUMMARY']]
df.STORY = 'summarize: ' + df.STORY
df['text']=df['STORY']
df['ctext']=df['SUMMARY']
print(df.head())
df=df.head(100)

                                               STORY  ...                                              ctext
0  summarize: Business Wire IndiaCSS Corp, a glob...  ...  CSS Corp, a global customer experience and tec...
1  summarize: Business Wire IndiaSmiths Detection...  ...                                                NaN
2  summarize: Business Wire India ECOVACS is pres...  ...  ECOVACS ROBOTICS, a market leader in innovativ...
3  summarize: Business Wire India  New analyses f...  ...  New analyses from pivotal BAVENCIOÂ® study rei...
4  summarize: Business Wire IndiaTakeda Pharmaceu...  ...            Takeda Pharmaceutical Company Limited (

[5 rows x 4 columns]


In [ ]:
import pandas as pd
tokenizer = T5Tokenizer.from_pretrained("t5-small")
  
  
df = pd.read_csv('/content/summary-dataset.csv',encoding='latin-1',delimiter="$")
df = df[['NEWS','SUMMARY']]
df.NEWS = 'summarize: ' + df.NEWS
df['text']=df['NEWS']
df['ctext']=df['SUMMARY']
print(df.head())
df=df.head(10)

                                                NEWS  ...                                              ctext
0  summarize: Job cuts 'false economy'  - TUC  Pl...  ...  Plans to shed 71,000 civil service jobs will p...
1  summarize: Probe launched on Ken Nazi jibe  An...  ...  Ms Gavron said she thought Mr Livingstone's co...
2  summarize: Blair blasts Tory spending plans  T...  ...  In his speech, Mr Blair contrasted a reformed ...
3  summarize: Iraq advice claim sparks new row  T...  ...  "After that Downing Street proceeded to set ou...
4  summarize: Women MPs reveal sexist taunts  Wom...  ...  But she said there was a difference between th...

[5 rows x 4 columns]


In [ ]:
from rouge_score import rouge_scorer

In [ ]:


def main():

    TRAIN_BATCH_SIZE = 5    # input batch size for training (default: 64)
    VALID_BATCH_SIZE = 5    # input batch size for testing (default: 1000)
    TRAIN_EPOCHS = 2   # number of epochs to train (default: 10)
    VAL_EPOCHS = 1 
    LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    SEED = 42               # random seed (default: 42)
    MAX_LEN = 512
    SUMMARY_LEN = 150 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(SEED) # pytorch random seed
    np.random.seed(SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    #print(str(tokenizer))
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
  

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size, random_state = SEED).reset_index(drop=True)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)


    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
  #  model = pd.read_pickle(r'/content/custom-T5')
    model = model.to("cpu")

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

    # Training loop
    print('Initiating Re-Training for the model on our dataset')
    
    for epoch in range(TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    return epoch,tokenizer,model,device,val_loader

    



In [ ]:
epoch,tokenizer,model,device,val_loader=main()

FULL Dataset: (100, 4)
TRAIN Dataset: (80, 4)
TEST Dataset: (20, 4)
Initiating Re-Training for the model on our dataset


NameError: ignored

In [ ]:
def getSummarisedText(data):
    text=data
    input_ids = tokenizer.encode(text, return_tensors='pt')
    # Generating summary ids
    summary_ids = bert_from_pickle.generate(
        input_ids=input_ids,

        max_length=150,
        num_beams=2,
        repetition_penalty=3.5,
        length_penalty=1.0,
        early_stopping=True
    )
    # Decoding the tensor and printing the summary.
    t5_summary = tokenizer.decode(summary_ids[0])
    return t5_summary

In [ ]:
actuals= df["SUMMARY"].tolist()
print(str(actuals))
print(len(actuals))
predicted=[]
maintext=df['NEWS'].tolist()

['Plans to shed 71,000 civil service jobs will prove to be a "false economy" that could hamper public sector reforms, according to a TUC report."Public services are improving but looking for simple savings through job cuts at this stage could be a false economy."They may shoot a Tory fox, but cutting thousands of civil service jobs will hit the morale and capabilities of the public servants expected to implement government reforms.Public and Commercial Services union members have already voted to strike over cuts for one day on 5 November.General secretary Brendan Barber warned the "costs could easily outweigh the benefits".The costs could easily outweigh the benefits."', 'Ms Gavron said she thought Mr Livingstone\'s comments were "inappropriate" but she did not believe the mayor was anti-Semitic.Speaking after the investigation was announced Bob Neill, leader of the London Assembly Conservatives, said: "He has behaved in a manner unbecoming of his office and in so doing, has shown ext

In [ ]:
bert_from_pickle = pd.read_pickle(r'/content/custom-T5')
#bert_from_pickle= T5ForConditionalGeneration.from_pretrained("t5-small")
for item in maintext:
    summary=getSummarisedText(item)
    print(summary)
    predicted.append(summary)



<pad> <extra_id_0> jobs 'false economy' - TUC Plans to shed 71,000 civil service jobs will prove to be a "false economy" that could hamper public sector reforms, according a TUC report. The TUC said cuts would deliver less than 6% of the <unk> £22bn ministers hope to save through efficiency reforms, according to a TUC report. Next Friday's strike action by the PCS is the biggest in the civil service since 1993, hitting Jobcentres, benefit agencies</s>
<pad> <extra_id_0>, he is in no way anti-Jewish, I wouldn't for a moment work with him if he were. The probe follows the London mayor's comments to a Jewish journalist comparing him to a concentration camp guard, after a party about a week ago. The local government watchdog also said the allegation related to a failure to respect others. It has the power to suspend or bar Labour's Mr Livingstone from office. On Sunday, Deputy Mayor Nicky Gavron</s>
<pad> <extra_id_0> plans Tony Blair has launched an attack on Conservative spending plans, 

In [ ]:
i=0
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
final_df = pd.DataFrame(columns=['Generated Text','Actual Text','Rogue Score'])
for prediction,actual in zip(predicted,actuals):
  try:
   print(prediction)
  #print(actual)
   scores = scorer.score(actual,
                      prediction)
   final_df=final_df.append({'Generated Text':prediction,'Actual Text':actual,'Rogue Score':scores},ignore_index=True)
   i+=1
  except Exception as e:
    print("ERROR")
final_df.to_csv('custompredictionspretrained.csv')

<pad> <extra_id_0> 'false economy' - TUC Plans to shed 71,000 civil service jobs will prove to be a "false economy" plans to shed 71,000 civil service jobs will prove to be a "false economy" that could hamper public sector reforms.</s>
<pad> <extra_id_0> :summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize:summarize</s>
<pad> Blair blasts conservative spending plans, saying they are a "ludicrous improbability" the prime minister has said the tories would cut <unk> £35bn in "wasteful" spending to allow <unk> £4bn in tax cuts.</s>
<pad> <extra_id_0> a statement about Lord Goldsmith's position was published in his new book, Lawless World. the tories say ministers must respond in Parliament to claims that the legal advice used to justify the Iraq war was drawn up at Number 10

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(1):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        
print('Output Files generated for review')

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0


AttributeError: ignored

In [ ]:
i=0
final_df = pd.DataFrame(columns=['Generated Text','Actual Text','Rogue Score'])
for prediction in predictions:
  scores = scorer.score(actuals[i],
                      prediction)
  final_df=final_df.append({'Generated Text':prediction,'Actual Text':actuals[i],'Rogue Score':scores},ignore_index=True)
  i+=1
final_df.to_csv('custompredictions.csv')

In [ ]:
!pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp37-none-any.whl size=6007 sha256=c1aec4d43993cdb0b4783add348e1d8c0585b427872f82cb98e2347c5e1194f9
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin


In [ ]:
import pickle
my_file = open("/content/custom-T5", 'wb')
my_file = pickle.dump(model,my_file)

/usr/local/lib/python3.7/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
#bert_from_pickle = pickle.load(my_file)
bert_from_pickle = pd.read_pickle(r'/content/custom-T5')

for epoch in range(1):
        predictions, actuals = validate(epoch, tokenizer, bert_from_pickle, device, val_loader)
 


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


KeyboardInterrupt: ignored

In [ ]:
def predict(epoch, tokenizer, model, device, loader):
    model.
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
# Concatenating the word "summarize:" to raw text
text2 = "summarize:" + "The all-party meeting called by Prime Minister Narendra Modi on June 24 is a major step towards the restoration of Jammu & Kashmir's statehood, a report said on June 20. Among those invited for the meeting include former J&K chief ministers Farooq Abdullah, Omar Abdullah, Mehbooba Mufti and Ghulam Nabi Azad.The CNN-News 18 report, which cited official sources, claimed that Modi would be discussing the roadmap for restoring J&K's statehood with the representatives of all political parties to be present at the meeting.Discussions, however, have been ruled out on restoring Article 370 - the now-abrogated law which granted J&K special status. The law was struck down on August 5, 2019, and the erstwhile state was bifurcated into the two union territories of Jammu & Kashmir and Ladakh.The status of Ladakh as a union territory will remain unchanged, the report said."
#print(text2)
# encoding the input text
input_ids=tokenizer.encode(text2, return_tensors='pt')
# Generating summary ids
summary_ids = bert_from_pickle.generate(
                input_ids = input_ids,
                
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
# Decoding the tensor and printing the summary.
t5_summary = tokenizer.decode(summary_ids[0])
print(t5_summary)

<pad> the all-party meeting called by Prime Minister Narendra Modi on June 24 is a major step towards the restoration of Jammu & Kashmir's statehood, a report said on June 20. Among those invited for the meeting include former J&K chief ministers Farooq Abdullah, Omar Abdullah, Mehbooba Mufti and Ghulam Nabi Azad.</s>
